In [51]:
#####################
##projection_dist
#####################
import struct
import numpy as np

N_CLASSES=10
DIM=384
ALPHA=0.5
BETA=0.95

l_pat=[2535,1691,1658,1736,1188,1996,1977,1410,2366,1428]
t_pat=[1485,2392,1939,2043,2403,1641,1831,2056,986,1140]


#[N_CLASSES][DIM]
fv_mean=[]
for i in range(N_CLASSES):
    tmp=[]
    for j in range(DIM):
        tmp.append(0)
    fv_mean.append(tmp)

#[N_CLASSES][DIM][DIM]
fv_cov=[]
for i in range(N_CLASSES):
    t1=[]
    for j in range(DIM):
        t2=[]
        for k in range(DIM):
            t2.append(0)
        t1.append(t2)
    fv_cov.append(t1)

#[N_CLASSES][DIM]
eig_index=[]
eigval=[]
for i in range(N_CLASSES):
    tmp=[]
    for j in range(DIM):
        tmp.append(0)
    eigval.append(tmp)
    eig_index.append(tmp)

#[N_CLASSES][DIM][DIM]
eigvec=[]
for i in range(N_CLASSES):
    t1=[]
    for j in range(DIM):
        t2=[]
        for k in range(DIM):
            t2.append(0)
        t1.append(t2)
    eigvec.append(t1)

#[N_CLASSES]
cutdim=[]
for x in range(N_CLASSES):
    cutdim.append(0)
    
def cut_dimension(value):
    S_dim=np.sum(value)
    base=BETA*S_dim
    S_cut=0.0
    for k in range(DIM):
        if(S_cut>=base):
            n=k
            break
        S_cut+=value[k]
    return n

def calc_projection_dist(fv,k,sigma2,alpha,dim):
    #calculation of the first term
    dist1=0.0
    for i in range(0,DIM):
        dist1+=(fv[i]-fv_mean[k][i])*(fv[i]-fv_mean[k][i])
    
    #calculation of the second term
    dist2=0.0
    factor=alpha/(1.0-alpha)*sigma2
    for j in range(0,cutdim[k]):
        SUM=0.0
        for i in range(0,DIM):
            SUM+=eigvec[k][i][j]*(fv[i]-fv_mean[k][i])
        SUM=SUM*SUM
        dist2+=eigval[k][j]/(eigval[k][j]+factor)*SUM
    
    return (dist1-dist2)

def swap(a,b):
    temp=a
    a=b
    b=a

def bsort_ascend(array,label,N_C):
    flag=True
    while flag:
        flag=False
        for i in range(len(array)-1):
            if(array[i]>array[i+1]):
                swap(array[i],array[i+1])
                flag=True
    return array

dist=[]
label=[]
lfv=[]
tfv=[]

for i in range(0,N_CLASSES):
    tmp=[]
    for l_serial in range(l_pat[i]):
        t2=[]
        for d in range(DIM):
            t2.append(0)
        tmp.append(t2)
    lfv.append(tmp)

for i in range(0,N_CLASSES):
    tmp=[]
    for t_serial in range(t_pat[i]):
        t2=[]
        for d in range(DIM):
            t2.append(0)
        tmp.append(t2)
    tfv.append(tmp)

for i in range(N_CLASSES):
    dist.append(0)
    label.append(0)

for l_digit in range(0,N_CLASSES):
    for l_serial in range(0,l_pat[l_digit]):
        fl_name="./fv_dir/lfv{}".format(l_digit)+"_{0:04d}".format(l_serial+1)
        fl=open(fl_name,"rb")
        for x in range(DIM):
            lfv[l_digit][l_serial][x]=list(struct.unpack('d',fl.read(8)))[0]
        fl.close()
        
for t_digit in range(0,N_CLASSES):
    for t_serial in range(0,t_pat[t_digit]):
        ft_name="./fv_dir/tfv{}".format(t_digit)+"_{0:04d}".format(t_serial+1)
        ft=open(ft_name,"rb")
        for x in range(DIM):
            tfv[t_digit][t_serial][x]=list(struct.unpack('d',ft.read(8)))[0]
        ft.close()

#calculation of fv_mean
for l_digit in range(0,N_CLASSES):
    fv_mean[l_digit]=np.average(lfv[l_digit],axis=0)[:]

#caluclation of fv_cov
for l_digit in range(0,N_CLASSES):
    fv_cov[l_digit]=np.cov(lfv[l_digit])[:]

#calculation of eigval,eigvec
for l_digit in range(0,N_CLASSES):
    eigval[l_digit],eigvec[l_digit]=np.linalg.eig(fv_cov[l_digit])
    np.sort(eigval[l_digit])[::-1]
    eig_index[l_digit]=np.argsort(eigval[l_digit])[::-1][:]

#cut dimension
cut=[]
for l_digit in range(0,N_CLASSES):
    cut.append(cut_dimension(eigval[l_digit]))
    print("l_digit={} : cut={}".format(l_digit,cut[l_digit]))

#calculation of average variance of feature vector elements
sigma2=0.0
for k in range(0,N_CLASSES):
    for i in range(0,DIM):
        sigma2+=fv_cov[k][i][i]

sigma2/=N_CLASSES*DIM

count=[]
result_index=[]
for l_digit in range(0,N_CLASSES):
    hit=[]
    for i in range(0,N_CLASSES):
        hit.append(0)
    for l_serial in range(0,l_pat[l_digit]):
        #calculation of modified projection distances
        for k in range(0,N_CLASSES):
            dist[k]=calc_projection_dist(lfv[l_digit][l_serial],k,sigma2,ALPHA,DIM)
        #sorting of distances in the ascending order
        #bsort_ascend(dist,label,N_CLASSES)
        sorted_dist=np.sort(dist)
        label=np.argsort(dist)
        hit[label[0]]+=1
    count.append(hit)
    result_index.append(np.argsort(hit)[::-1])

count_test=[]
result_test_index=[]
for t_digit in range(0,N_CLASSES):
    hit=[]
    for i in range(0,N_CLASSES):
        hit.append(0)
    for t_serial in range(0,t_pat[t_digit]):
        #calculation of modified projection distances
        for k in range(0,N_CLASSES):
            dist[k]=calc_projection_dist(tfv[t_digit][t_serial],k,sigma2,ALPHA,DIM)
        #sorting of distances in the ascending order
        #bsort_ascend(dist,label,N_CLASSES)
        sorted_dist=np.sort(dist)
        label=np.argsort(dist)
        hit[label[0]]+=1
    count_test.append(hit)
    result_test_index.append(np.argsort(hit)[::-1])

acum=[]
for digit in range(0,N_CLASSES):
    tmp=[]
    S=0.0
    for rank in range(0,N_CLASSES):
        S+=count[digit][result_index[digit][rank]]
        tmp.append(S*100.0/np.sum(count[digit]))
    acum.append(tmp)

l_total=[]
S=0.0
for rank in range(0,N_CLASSES):
    for digit in range(0,N_CLASSES):
        S+=count[digit][result_index[digit][rank]]
    l_total.append(S*100.0/np.sum(l_pat))
        
acum_t=[]
for digit in range(0,N_CLASSES):
    tmp=[]
    S=0.0
    for rank in range(0,N_CLASSES):
        S+=count_test[digit][result_test_index[digit][rank]]
        tmp.append(S*100.0/np.sum(count_test[digit]))
    acum_t.append(tmp)

t_total=[]
S=0.0
for rank in range(0,N_CLASSES):
    for digit in range(0,N_CLASSES):
        S+=count_test[digit][result_test_index[digit][rank]]
    t_total.append(S*100.0/np.sum(t_pat))

#display of recognition result
print("-- Recognition result --")
for d in range(0,N_CLASSES):
    print(d)
    print("{:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}".format(acum[d][0],acum[d][1],acum[d][2],acum[d][3],acum[d][4],acum[d][5],acum[d][6],acum[d][7],acum[d][8],acum[d][9]))
    print("{:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}".format(acum_t[d][0],acum_t[d][1],acum_t[d][2],acum_t[d][3],acum_t[d][4],acum_t[d][5],acum_t[d][6],acum_t[d][7],acum_t[d][8],acum_t[d][9]))
    print()
print("Total")
print("{:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}".format(l_total[0],l_total[1],l_total[2],l_total[3],l_total[4],l_total[5],l_total[6],l_total[7],l_total[8],l_total[9]))
print("{:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}".format(t_total[0],t_total[1],t_total[2],t_total[3],t_total[4],t_total[5],t_total[6],t_total[7],t_total[8],t_total[9]))


l_digit=0 : cut=85
l_digit=1 : cut=65
l_digit=2 : cut=125
l_digit=3 : cut=122
l_digit=4 : cut=143
l_digit=5 : cut=117
l_digit=6 : cut=94
l_digit=7 : cut=82
l_digit=8 : cut=113
l_digit=9 : cut=90
-- Recognition result --
0
93.89      95.98      97.67      98.58      99.37      99.57      99.72      99.84      99.92      100.00
94.81      96.30      97.58      98.86      99.60      99.73      99.87      99.93      100.00      100.00

1
97.99      98.76      99.47      99.70      99.88      99.94      100.00      100.00      100.00      100.00
98.24      98.91      99.21      99.41      99.58      99.75      99.87      99.96      100.00      100.00

2
93.61      95.96      98.01      98.61      99.10      99.46      99.76      99.88      100.00      100.00
90.97      95.20      98.14      98.81      99.33      99.64      99.85      100.00      100.00      100.00

3
91.13      93.43      95.68      97.00      98.16      98.85      99.48      99.77      99.94      100.00
89.04      92.02   